# Middleware: Human In The Loop
<img src="./assets/LC_HITL.png" width="300">



## Setup

In [12]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [13]:
from langchain_core.tools import tool


@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [14]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.
"""

In [15]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "execute_sql": {
                    "allow_accept": True,
                    "allow_respond": True,
                    "allow_edit": False
                },
            },
            description_prefix="Tool execution pending approval",
        ),
    ],
)

In [16]:
from langgraph.types import Command

question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

if "__interrupt__" in result:
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{result['__interrupt__'][-1].value[-1]['description']}\033[0m"
    )

    result = agent.invoke(
        Command(
            # resume=[{"type": "accept"}]
            resume=[
                {"type": "response", "args": "the database is offline."}
            ]  # or "edit", "respond"
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution pending approval

Tool: execute_sql
Args: {'query': 'SELECT name FROM employees ORDER BY name LIMIT 5;'}
--------------------------------------------------------------------------------
The database is offline. Please try again later.


In [18]:
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
)

while "__interrupt__" in result:
    result = agent.invoke(
        Command(
            resume=[{"type": "accept"}]
        ),
        config=config,  # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

for msg in result["messages"]:
    msg.pretty_print()

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_EdAv89wBlSnfVKwzNEQinVMP)
 Call ID: call_EdAv89wBlSnfVKwzNEQinVMP
  Args:
    query: SELECT name, sql FROM sqlite_master WHERE type='table' AND lower(name) LIKE '%employee%';
================================= Tool Message =================================
Name: execute_sql

[('Employee', 'CREATE TABLE [Employee]\n(\n    [EmployeeId] INTEGER  NOT NULL,\n    [LastName] NVARCHAR(20)  NOT NULL,\n    [FirstName] NVARCHAR(20)  NOT NULL,\n    [Title] NVARCHAR(30),\n    [ReportsTo] INTEGER,\n    [BirthDate] DATETIME,\n    [HireDate] DATETIME,\n    [Address] NVARCHAR(70),\n    [City] NVARCHAR(40),\n...')